In [1]:
!pip install kaggle==1.5.12

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions

In [3]:
!mkdir datasets

In [4]:
import shutil

In [5]:
source_folder = 'checkpoints'
destination_folder = 'datasets/checkpoints'  # Create 'checkpoints' inside 'datasets'

shutil.move(source_folder, destination_folder)

In [6]:
!kaggle datasets download -d mozillaorg/common-voice -p /content/datasets --force

In [7]:
%cd datasets

/content/datasets


In [8]:
!unzip common-voice.zip

In [9]:
import pandas as pd

In [10]:
dev_df = pd.read_csv('cv-valid-train.csv')
# dev_df.head()

In [11]:
# dev_df.info()

In [12]:
# dev_df = dev_df

In [13]:
# from IPython.display import Audio, display
# import os

# def display_audio(audio_path):
#   display(Audio(audio_path))

# audio_directory = 'cv-valid-dev/cv-valid-dev'
# audio_files = [audio for audio in os.listdir(audio_directory)]

# for audio_file in audio_files[2000:2050]:
#   audio_path = os.path.join(audio_directory, audio_file)
#   display_audio(audio_path)


In [14]:
# dev_df['down_votes'].value_counts()

In [15]:
# dev_df[dev_df['down_votes'] == 0][:10]

 => It seems that the difference between upvotes and downvotes doesn't relate to the quality of audios

**Preprocessing steps**



In [16]:
dev_df = dev_df.drop(columns=dev_df.columns[dev_df.columns.get_loc('up_votes') : dev_df.columns.get_loc('duration') + 1])

In [17]:
# dev_df.head()

In [18]:
import torch
import os
import torchaudio
import warnings
warnings.filterwarnings('ignore')

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [20]:
audio_directory = 'cv-valid-train'

train_size = int(0.8 * len(dev_df))
train_df = dev_df.iloc[:train_size]
test_df = dev_df.iloc[train_size+1:]

train_audio_files = [os.path.join(f) for f in train_df['filename']]
train_texts = train_df['text'].tolist()

test_audio_files = [os.path.join(f) for f in test_df['filename']]
test_texts = test_df['text'].tolist()

In [21]:
# import torchaudio
# import librosa
# import os

# durations = 0
# for audio_file in train_audio_files[:80000]:
#   audio_data, sr = librosa.load(os.path.join(audio_directory, audio_file), sr=None)
#   # ===== SILENCE REMOVAL =====
#   original_duration = len(audio_data) / sr
#   audio_data = remove_silence(audio_data, sr)
#   new_duration = len(audio_data) / sr
#   silence_removed = original_duration - new_duration

#   # ===== NOISE REDUCTION =====
#   # Only attempt noise reduction if we have enough audio left
#   if len(audio_data) > 0.2 * sr:  # At least 200ms remaining
#       # Estimate noise from non-silent portions (first 100ms after silence removal)
#       noise_samples = int(0.1 * sr)
#       noise_profile = audio_data[:min(noise_samples, len(audio_data))]

#       reduced_noise = nr.reduce_noise(
#           y=audio_data,
#           sr=sr,
#           y_noise=noise_profile,
#           stationary=False,
#           prop_decrease=0.8
#       )
#   else:
#       reduced_noise = audio_data  # Skip if too short after silence removal

#   # ===== VOLUME NORMALIZATION =====
#   rms = np.sqrt(np.mean(reduced_noise**2))
#   current_dBFS = 20 * np.log10(max(rms, 1e-10))
#   target_dBFS = -20
#   gain = min(target_dBFS - current_dBFS, 20)

#   if gain > 1:
#       amplified_audio = reduced_noise * (10 ** (gain / 20))
#       amplified_audio = np.clip(amplified_audio, -1.0, 1.0)
#   else:
#       amplified_audio = reduced_noise

#   # ===== RESAMPLING =====
#   target_sr = 16000
#   if sr != target_sr:
#       resampled_audio = librosa.resample(amplified_audio, orig_sr=sr, target_sr=target_sr)
#   else:
#       resampled_audio = amplified_audio


#   durations += librosa.get_duration(y=resampled_audio, sr=target_sr)

# print("Durations: ", durations)


In [22]:
import re

In [23]:
def clean_text(text):
    text = text.lower().strip()

    # Handle apostrophes/contractions carefully
    text = re.sub(r"([!\#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text)
    text = re.sub("[^A-Za-z0-9]+", " ", text)
    text = re.sub(" +", " ", text)

    return text

In [24]:
from tokenizers import Tokenizer, models, trainers
from tokenizers import pre_tokenizers

In [25]:
# Initialize with byte-level BPE (better for names/contractions)
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# Trainer with larger vocab
trainer = trainers.BpeTrainer(
    special_tokens=["<pad>", "<unk>", "<bos>", "<eos>", "<blank>"],
    vocab_size=8000,
    min_frequency=2,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()  # Better for special characters
)

# Train on properly cleaned text
tokenizer.train_from_iterator([clean_text(t) for t in train_texts], trainer)
tokenizer.save("cv_tokenizer.json")

In [26]:
tokenizer.get_vocab_size()

8000

In [27]:
!pip install noisereduce

In [28]:
import librosa
import soundfile as sf
import noisereduce as nr
from IPython.display import Audio, display
import numpy as np

In [29]:
def remove_silence(audio, sr, top_db=20, frame_length=2048, hop_length=512):
    """Remove silent segments from audio using librosa's trim function"""
    # Trim leading and trailing silence
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=top_db,
                                          frame_length=frame_length,
                                          hop_length=hop_length)
    return trimmed_audio

In [30]:
def preprocess_for_rnnt_torchaudio(file_paths, target_sr=16000, max_duration=5, device='cuda'):
    """GPU-accelerated batch preprocessing pipeline for RNN-T
    Returns:
        log_mel: (batch_size, T, 80) tensor of log Mel spectrograms
        lengths: (batch_size,) tensor of actual lengths in frames
    """
    # Initialize mel transform once
    mel_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=target_sr,
        n_fft=400,
        hop_length=160,
        win_length=400,
        n_mels=80,
    ).to(device)

    mel_specs = []
    audio_lengths = []

    # Calculate max_len in frames based on max_duration
    max_len = int(max_duration * target_sr / 160)  # hop_length is 160

    for file_path in file_paths:
        # Load audio with original sample rate
        audio_data, sr = librosa.load(file_path, sr=None)

        # ===== SILENCE REMOVAL =====
        original_duration = len(audio_data) / sr
        audio_data = remove_silence(audio_data, sr)
        new_duration = len(audio_data) / sr
        silence_removed = original_duration - new_duration

        # ===== NOISE REDUCTION =====
        # Only attempt noise reduction if we have enough audio left
        if len(audio_data) > 0.2 * sr:  # At least 200ms remaining
            # Estimate noise from non-silent portions (first 100ms after silence removal)
            noise_samples = int(0.1 * sr)
            noise_profile = audio_data[:min(noise_samples, len(audio_data))]

            reduced_noise = nr.reduce_noise(
                y=audio_data,
                sr=sr,
                y_noise=noise_profile,
                stationary=False,
                prop_decrease=0.8
            )
        else:
            reduced_noise = audio_data  # Skip if too short after silence removal

        # ===== VOLUME NORMALIZATION =====
        rms = np.sqrt(np.mean(reduced_noise**2))
        current_dBFS = 20 * np.log10(max(rms, 1e-10))
        target_dBFS = -20
        gain = min(target_dBFS - current_dBFS, 20)

        if gain > 1:
            amplified_audio = reduced_noise * (10 ** (gain / 20))
            amplified_audio = np.clip(amplified_audio, -1.0, 1.0)
        else:
            amplified_audio = reduced_noise

        # ===== RESAMPLING =====
        target_sr = 16000
        if sr != target_sr:
            resampled_audio = librosa.resample(amplified_audio, orig_sr=sr, target_sr=target_sr)
        else:
            resampled_audio = amplified_audio

        sf.write(file_path, resampled_audio, target_sr, format='mp3')
        audio_tensor, sr = torchaudio.load(file_path)
        audio_tensor = audio_tensor.to(device)
        # 3. Mel transform
        mel_spec = mel_transform(audio_tensor)  # (channels, n_mels, time)
        log_mel = torchaudio.transforms.AmplitudeToDB()(mel_spec)
        log_mel = (log_mel - log_mel.min()) / (log_mel.max() - log_mel.min())
        # Permute to (channels, time, n_mels)
        log_mel = log_mel.permute(0, 2, 1)
        # 4. Padding/trimming
        if log_mel.size(1) > max_len:
            log_mel = log_mel[:, :max_len+1, :]
        else:
            pad_amount = max_len + 1 - log_mel.size(1)
            log_mel = torch.nn.functional.pad(log_mel, (0, 0, 0, pad_amount))

        mel_specs.append(log_mel.squeeze(0))  # Remove channel dimension if single channel
        audio_lengths.append(log_mel.size(1))

    # Stack into batch (batch_size, T, n_mels)
    mel_specs = torch.stack(mel_specs)
    audio_lengths = torch.tensor(audio_lengths, dtype=torch.int32)

    return mel_specs, audio_lengths

In [31]:
# import matplotlib.pyplot as plt

In [32]:
# log_mel, audio_lengths = preprocess_for_rnnt_torchaudio(['cv-valid-test/cv-valid-test/sample-000001.mp3'], device=device)
# plt.figure(figsize=(10, 4))
# # Convert the PyTorch tensor to a NumPy array
# librosa.display.specshow(
#     log_mel.cpu().numpy()[0], # Access the first element if it's a batch of size 1
#     # sr=16000,
#     x_axis='time',
#     y_axis='mel',
#     fmax=8000,
# )
# plt.colorbar(format='%+2.0f dB')
# plt.title('Mel Spectrogram')
# plt.tight_layout()
# plt.show()

In [33]:
class Encoder(torch.nn.Module):
    def __init__(self, input_dim=80, hidden_dim=256, num_layers=3):
        super().__init__()
        self.lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=False,
            batch_first=True,
            dropout=0.2
        )
        self.linear = torch.nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        # x: (B, T, input_dim)
        x, _ = self.lstm(x)  # (B, T, H)
        x = self.linear(x)   # (B, T, H)
        return x

class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            dropout=0.2)
        self.linear = torch.nn.Linear(hidden_dim, hidden_dim)

    def forward(self, y):
        # y: (B, U)
        y = self.embed(y)     # (B, U, E)
        y, _ = self.lstm(y)   # (B, U, H)
        y = self.linear(y)    # (B, U, H)
        return y

class JointNetwork(torch.nn.Module):
    def __init__(self, hidden_dim, vocab_size):
        super().__init__()
        # Use a more robust approach with a multi-layer network
        self.joint = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),
            torch.nn.Tanh(),
            torch.nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, h_enc, h_dec):
        # h_enc: (B, T, H), h_dec: (B, U, H)

        # Get batch size and sequence lengths
        batch_size = h_enc.size(0)
        T = h_enc.size(1)  # Audio sequence length
        U = h_dec.size(1)  # Text sequence length

        # Expand dimensions for broadcasting
        h_enc = h_enc.unsqueeze(2)  # (B, T, 1, H)
        h_dec = h_dec.unsqueeze(1)  # (B, 1, U, H)

        # Expand to create the full cartesian product for alignment
        h_enc = h_enc.expand(-1, -1, U, -1)  # (B, T, U, H)
        h_dec = h_dec.expand(-1, T, -1, -1)  # (B, T, U, H)

        # Concatenate features
        joint = torch.cat([h_enc, h_dec], dim=-1)  # (B, T, U, 2H)

        # Apply joint network to get logits
        logits = self.joint(joint)  # (B, T, U, V)

        return logits

class RNNTransducer(torch.nn.Module):
    def __init__(self, vocab_size, encoder_dim=256, decoder_dim=256):
        super().__init__()
        self.encoder = Encoder(hidden_dim=encoder_dim)
        self.decoder = Decoder(vocab_size, hidden_dim=decoder_dim)
        self.joint = JointNetwork(decoder_dim, vocab_size)
        self.vocab_size = vocab_size

    def forward(self, x, y):
        # x: (B, T, features), y: (B, U)
        h_enc = self.encoder(x)  # (B, T, H)
        h_dec = self.decoder(y)  # (B, U, H)

        # Both should now have the same hidden dimension
        logits = self.joint(h_enc, h_dec)  # (B, T, U, V)
        return logits



# First, let's prepare the dataset class
class AudioTextDataset(torch.utils.data.Dataset):
    def __init__(self, audio_files, texts, tokenizer, max_text_length=100,
                 target_sr=16000, max_duration=5):
        """
        Dataset class that handles:
        - File paths for batch audio processing
        - Text sequences with tokenization
        - Automatic audio preprocessing in batches
        """
        self.audio_files = [str(f) for f in audio_files]  # Ensure string paths
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_text_length = max_text_length
        self.target_sr = target_sr
        self.max_duration = max_duration

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        """
        Returns single item with:
        - audio_file: Path for batch processing
        - text: Raw text string for tokenization in collate
        """
        return {
            'audio_file': self.audio_files[idx],
            'text': self.texts[idx],
            'tokenizer': self.tokenizer,  # Pass tokenizer for collate_fn
            'max_text_length': self.max_text_length
        }

def collate_fn(batch, device='cuda'):
    """
    Custom collate function that:
    1. Processes audio files in batch using preprocess_for_rnnt_torchaudio
    2. Tokenizes and pads text sequences
    3. Handles audio length preservation
    """
    # Extract batch components
    audio_files = [item['audio_file'] for item in batch]
    texts = [item['text'] for item in batch]

    # Batch process audio files
    audio_features, audio_lengths = preprocess_for_rnnt_torchaudio(
        audio_files,
        target_sr=batch[0].get('target_sr', 16000),
        max_duration=batch[0].get('max_duration', 5),
        device=device
    )  # (B, T, 80)

    target_lengths = [item['max_text_length'] for item in batch]

    # Tokenize and pad texts
    text_sequences = []
    for item in batch:
        encoding = item['tokenizer'].encode(item['text'])
        text_ids = encoding.ids

        # Pad or truncate
        max_len = item['max_text_length']
        if len(text_ids) < max_len:
            text_ids = text_ids + [0] * (max_len - len(text_ids))
        else:
            text_ids = text_ids[:max_len]

        text_sequences.append(torch.tensor(text_ids, dtype=torch.int32))

    target_lengths_tensor = torch.tensor(target_lengths, dtype=torch.int32)
    # Stack text sequences
    text_tensor = torch.stack(text_sequences)

    return {
        'audio': audio_features.to(device),  # (B, T, 80)
        'text': text_tensor.to(device),     # (B, U_max)
        'target_lengths': target_lengths_tensor.to(device),
        'audio_lengths': torch.tensor(audio_lengths, device=device) # (B,)
    }

In [34]:
!pip install jiwer

In [35]:
import torch.nn.functional as F
import jiwer

In [36]:
def train_epoch(model, dataloader, optimizer, scheduler, device, tokenizer, mixed_precision=True):
    model.train()
    total_loss = 0
    steps = 0

    # Enable mixed precision for faster training and lower memory usage
    scaler = torch.cuda.amp.GradScaler() if mixed_precision else None

    for batch_idx, batch in enumerate(dataloader):
        # Get batch data
        audio = batch['audio']  # (B, T, 80)
        text = batch['text']    # (B, U)

        # Get lengths
        input_lengths = batch['audio_lengths']
        target_lengths = batch['target_lengths']

        print("Input length: ", input_lengths)
        print("Target length: ", target_lengths)

        # Clear gradients
        optimizer.zero_grad()

        if mixed_precision:
            with torch.cuda.amp.autocast():
                # Forward pass - encoder
                encoder_output = model.encoder(audio)
                concat_text = F.pad(text, pad=(1, 0, 0, 0), value=0)
                # Forward pass - decoder and joint
                h_dec = model.decoder(concat_text)  # (B, U, H)
                logits = model.joint(encoder_output, h_dec)  # (B, T, U, V)

                print("Logits shape: ", logits.shape)
                # For targets, use labels shifted right (exclude BOS)
                targets = text.contiguous()  # Remove only BOS for targets
                print("Targets shape: ", targets.shape)
                # Compute loss
                loss = torchaudio.functional.rnnt_loss(
                    logits=torch.nn.functional.log_softmax(logits, dim=-1),
                    targets=targets,
                    logit_lengths=input_lengths,
                    target_lengths=target_lengths,
                    blank=0,
                    reduction='mean'
                )

            # Backward pass with scaled gradients
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)  # Lower clip threshold
            scaler.step(optimizer)
            scaler.update()
        else:
            # Standard training without mixed precision
            encoder_output = model.encoder(audio)
            concat_text = F.pad(text, pad=(1, 0, 0, 0), value=0)
            # Forward pass - decoder and joint
            h_dec = model.decoder(concat_text)  # (B, U, H)
            logits = model.joint(encoder_output, h_dec)  # (B, T, U, V)

            targets = text.contiguous()

            loss = torchaudio.functional.rnnt_loss(
                logits=torch.nn.functional.log_softmax(logits, dim=-1),
                targets=targets,
                logit_lengths=input_lengths,
                target_lengths=target_lengths,
                blank=0,
                reduction='mean'
            )

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

        # Update learning rate with scheduler
        if scheduler is not None:
            scheduler.step()

        total_loss += loss.item()
        steps += 1

    return total_loss / steps

In [37]:
def evaluate(model, dataloader, device, tokenizer):
    """
    Evaluate the model on the test set using Character Error Rate (CER)

    Args:
        model: RNNTransducer model
        dataloader: DataLoader containing test data
        device: Device to run evaluation on
        tokenizer: Tokenizer for decoding predictions

    Returns:
        float: Average CER across all samples
    """
    model.eval()
    total_cer = 0
    num_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            # Get batch data
            audio = batch['audio']  # (B, T, 80)
            text = batch['text']    # (B, U)

            # Get lengths
            input_lengths = batch['audio_lengths']
            target_lengths = batch['target_lengths']

            # Forward pass through encoder
            encoder_output = model.encoder(audio)  # (B, T, H)

            # Initialize decoder input with zero token
            zero_token = torch.zeros(1, 1, dtype=torch.long, device=device)

            def decode(enc_state, length):
                token_list = []

                # Initial decoder state
                dec_state, hidden = model.decoder(zero_token)

                for t in range(length):
                    # Get current encoder state
                    enc_t = enc_state[t].view(-1)  # (H,)
                    dec_t = dec_state.view(-1)     # (H,)

                    # Get joint network output
                    logits = model.joint(enc_t, dec_t)
                    probs = F.softmax(logits, dim=0)
                    pred = torch.argmax(probs, dim=0)
                    pred = int(pred.item())

                    # If not blank token (0), add to sequence and update decoder
                    if pred != 0:
                        token_list.append(pred)
                        token = torch.LongTensor([[pred]]).to(device)
                        dec_state, hidden = model.decoder(token, hidden=hidden)

                return token_list

            # Process each sample in batch
            results = []
            for i in range(audio.size(0)):
                decoded_seq = decode(encoder_output[i], input_lengths[i])
                results.append(decoded_seq)

            # Calculate CER for each prediction
            for i, pred_tokens in enumerate(results):
                # Convert predictions to text
                pred_text = tokenizer.decode(pred_tokens)
                true_text = tokenizer.decode(text[i])

                # Calculate CER
                cer = jiwer.character_error_rate(true_text, pred_text)
                total_cer += cer
                num_samples += 1

    return total_cer / num_samples

In [38]:
tokenizer = Tokenizer.from_file("cv_tokenizer.json")

In [39]:
# Create datasets
train_dataset = AudioTextDataset(
    audio_files=[os.path.join(audio_directory, f) for f in train_audio_files],
    texts=train_texts,
    tokenizer=tokenizer,
)

In [40]:
test_dataset = AudioTextDataset(
    audio_files=[os.path.join(audio_directory, f) for f in test_audio_files],
    texts=test_texts,
    tokenizer=tokenizer,
)

In [41]:
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        shuffle=True,
        collate_fn=lambda batch: collate_fn(batch, device=device)
    )

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda batch: collate_fn(batch, device=device)
)

In [42]:
# checkpoint_path = 'checkpoints/rnnt_epoch_94.pt'
# checkpoint = torch.load(checkpoint_path)

In [43]:
from typing import List, Union

In [44]:
# Initialize model
model = RNNTransducer(vocab_size=tokenizer.get_vocab_size()).to(device)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=5e-3,
    weight_decay=1e-6,
    betas=(0.9, 0.999)
)
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

save_dir = 'checkpoints'

# Training loop
for epoch in range(50):
    # Train for one epoch
    train_loss = train_epoch(
        model=model,
        dataloader=train_loader,
        optimizer=optimizer,
        scheduler=None,
        device=device,
        tokenizer=tokenizer,
        mixed_precision=True
    )

    # Evaluate on test set
    test_loss = evaluate(
        model=model,
        dataloader=test_loader,
        device=device,
        tokenizer=tokenizer
    )

    print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f}")

    # Save checkpoint
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        checkpoint_path = os.path.join(save_dir, f"rnnt_epoch_{epoch}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss,
        }, checkpoint_path)
        print(f"Saved checkpoint to {checkpoint_path}")

print("Training complete!")

Input length:  tensor([501], device='cuda:0', dtype=torch.int32)
Target length:  tensor([100], device='cuda:0', dtype=torch.int32)
Logits shape:  torch.Size([1, 501, 101, 8000])
Targets shape:  torch.Size([1, 100])
Input length:  tensor([501], device='cuda:0', dtype=torch.int32)
Target length:  tensor([100], device='cuda:0', dtype=torch.int32)
Logits shape:  torch.Size([1, 501, 101, 8000])
Targets shape:  torch.Size([1, 100])
Input length:  tensor([501], device='cuda:0', dtype=torch.int32)
Target length:  tensor([100], device='cuda:0', dtype=torch.int32)
Logits shape:  torch.Size([1, 501, 101, 8000])
Targets shape:  torch.Size([1, 100])
Input length:  tensor([501], device='cuda:0', dtype=torch.int32)
Target length:  tensor([100], device='cuda:0', dtype=torch.int32)
Logits shape:  torch.Size([1, 501, 101, 8000])
Targets shape:  torch.Size([1, 100])
Input length:  tensor([501], device='cuda:0', dtype=torch.int32)
Target length:  tensor([100], device='cuda:0', dtype=torch.int32)
Logits sh

KeyboardInterrupt: 

In [ ]:
!zip -r checkpoints.zip checkpoints

In [ ]:
from google.colab import files
files.download('checkpoints.zip')